In [ ]:
import numpy as np
import pandas as pd
import nd2reader
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews.operation.datashader import regrid
import skimage.filters
import skimage.feature
import scipy.ndimage
import peakutils
from tqdm.autonotebook import tqdm
import dask
import dask.array as da
import distributed
from distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
from cytoolz import partial, compose, juxt
from itertools import repeat
from glob import glob
import cachetools
import numpy_indexed
import pickle
import pyarrow as pa
import warnings
import os
from numbers import Integral
from dask.delayed import Delayed

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from segmentation import *
# from util import *
# from matriarch_stub import *
import segmentation
import matriarch_stub

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
hv.extension("bokeh")

# Run

In [ ]:
# dask.config.config['distributed']['scheduler']['allowed-failures'] = 20
# dask.config.config['distributed']['worker']['memory'] = {'target': 0.4,
#                                                         'spill': 0.5,
#                                                         'pause': 0.9,
#                                                         'terminate': 0.95}

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="03:00:00",
    memory="8GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/projects/molecule-counting/log",
    cores=1,
    processes=1,
)
# diagnostics_port=('127.0.0.1', 8787),
# env_extra=['export PYTHONPATH=\"/home/jqs1/projects/matriarch\"'])
client = Client(cluster)

In [ ]:
cluster

In [ ]:
cluster.scale(0)

# Heterogenous cluster

In [ ]:
# TODO
# fix autothresholding
# break up segment into sub-functions
# handwrite frangi that works with float32, uint16 (github issue for float32 bug)

# memory profile regionprops

# tree() that records insertion times, utility func to turn into benchmarking for each step (hierarchical inter-step times)
# split photobleaching task into multiple sub-tasks (each is a single-threaded .compute() call)
# pin segmentation/regionprops tasks to high-RAM nodes (in heterogenous dask cluster)

# filter by FOCUS (???)

# named_funcs_as_juxt: decorator to turn {'func1': func1, ('q0.5', 'q0.7'): partial(np.percentile, q=(0.5,0.7))} into a multiple-valued func
# zarrification of labels (skip??)

# pass in frame metadata to filter funcs (requires unified metadata representation)

## Run

In [ ]:
def segmentation_frame_filter(img):
    return True


def segmentation_labels_filter(labels, img):
    return labels.max() < 10000

In [ ]:
base_filename = "/n/scratch2/jqs1"
# filenames = glob(os.path.join(base_filename, '190922/*/*photobleaching*.nd2'))

In [ ]:
# TODO: try segmenting everything in phase (to reduce bias of segmenting in different channels)
# G_GR  G_RG  G-R_GR  G-R_RG  R-G_GR  R_GR  R-G_RG  R_RG
seg_channel_to_files = {
    "RFP-PENTA": ["191312/R_RG/*.nd2"],
    #'191312/R_GR/*.nd2'], # missing GFP-PENTA for all but one
    "GFP-PENTA": [
        "191312/G_GR/*.nd2",
        "191312/G_RG/*.nd2",
        "191312/G-R_GR/GR*.nd2",  # missing RFP-PENTA for all but two
        "191312/G-R_RG/*.nd2",
        "191312/R-G_GR/*.nd2",
        "191312/R-G_RG/*.nd2",
    ],
}

In [ ]:
# segmentation.cluster_nd2_by_positions(glob(os.path.join(base_filename, '191312/R-G_GR/*.nd2')), ignored_channels=['BF'])

In [ ]:
funcs = None  # not used

In [ ]:
data_graph = {}
for segmentation_channel, file_patterns in tqdm(seg_channel_to_files.items()):
    for file_pattern in file_patterns:
        data_graph[file_pattern] = {}
        filenames = glob(os.path.join(base_filename, file_pattern))
        clustered_filenames = segmentation.cluster_nd2_by_positions(
            filenames, ignored_channels=["BF"]
        )
        for channel_to_file in tqdm(clustered_filenames.values()):
            segmentation_filename = channel_to_file[segmentation_channel]
            channels = list(set(channel_to_file.keys()) - set(["BF"]))
            d = {}
            for channel in channels:
                d[channel] = segmentation.process_photobleaching_file(
                    funcs,
                    channel_to_file[channel],
                    photobleaching_channel=channel,
                    segmentation_filename=segmentation_filename,
                    segmentation_channel=segmentation_channel,
                    time_slice=slice(10),
                    rechunk=True,
                    segmentation_frame_filter=segmentation_frame_filter,
                    segmentation_labels_filter=segmentation_labels_filter,
                )
            rep = d[channels[0]][0]
            seg_data = {
                "segmentation_filename": segmentation_filename,
                "segmentation_channel": segmentation_channel,
                "segmentation_frame": rep["segmentation_frame"],
                "labels": rep["labels"],
                "traces": {channel: d[channel][0]["traces"] for channel in channels},
                "photobleaching_filenames": channel_to_file,
            }
            data_graph[file_pattern][segmentation_filename] = seg_data

In [ ]:
# split up computes so we can gather results from multiple workers
# (otherwise the single worker assembling the dict will run out of memory)
# TODO: use recursive_map(..., levels=?)
data_futures = {
    k: {k2: client.compute(v2) for k2, v2 in v.items()} for k, v in data_graph.items()
}

In [ ]:
del data_futures

## Save data

In [ ]:
data = client.gather(data_futures)

In [ ]:
filename = "/n/groups/paulsson/jqs1/molecule-counting/200107photobleaching.pickle"
with open(filename, "wb") as f:
    pickle.dump(data, f)

In [ ]:
import pickle

data = pickle.load(
    open("/n/groups/paulsson/jqs1/molecule-counting/200103photobleaching.pickle", "rb")
)

In [ ]:
{
    k: {pos: np.asarray(d["labels"]).max() for pos, d in v.items()}
    for k, v in data.items()
    if k[0] != "_"
}

In [ ]:
d = data["191312/R-G_RG/*.nd2"][
    "/n/scratch2/jqs1/191312/R-G_RG/RG_100pct_100ms_100pct_100ms.nd2_0011.nd2"
]

In [ ]:
d.keys()

In [ ]:
plt.imshow(d["segmentation_frame"])

In [ ]:
plt.imshow(matriarch_stub.permute_labels(d["labels"]))

In [ ]:
plt.plot(np.log(d["traces"]["GFP-PENTA"]["mean"].T))